## Projektarbeit Mühle

### Imports

In [31]:
import itertools
from collections import defaultdict
import PySimpleGUI as psGui
import numpy as np
import time
import copy
from tensorflow import keras
import tensorflow as tf
import datetime as dt
import sklearn.preprocessing as pre
from typing import List
import math
import multiprocessing as mp

### Class definitions

#### Environment

In [106]:

class MillEnv(object):
    def __init__(self):
        self.isPlaying: int = 1
        self.gamePhase: list = [0,0]
        self.moveNeeded: int = 0
        self.inHand: list = [9,9]
        self.onBoard: list = [0,0]
        self.checkerPositions: list = [[],[]]
        self.selected: int = -1
        self.board: np.ndarray = np.zeros(24)
        self.winner = 0
        self.columns: np.ndarray = np.array([[0,1,2], # for determining how many checkers from each player are in a row
                        [3,4,5],
                        [6,7,8],
                        [9,10,11],
                        [12,13,14],
                        [15,16,17],
                        [18,19,20],
                        [21,22,23],
                        [0,9,21],
                        [3,10,18],
                        [6,11,15],
                        [1,4,7],
                        [16,19,22],
                        [8,12,17],
                        [5,13,20],
                        [2,14,23],
                        ])
        self.previusStates = np.array([np.zeros(24)])
    def makeMove(self, move: int) -> (bool, int):
        if self.winner == 0:
            valid: bool = False
            last_state: tuple = self.getSummary(self.isPlaying)
            last_player: int = self.isPlaying
            reward: int = 0
            if self.moveNeeded == 0: # Set Checker on position
                if self.board[move] == 0:
                    self.board[move] = self.isPlaying
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].append(move)
                    valid = True
                    if self.gamePhase[1 if self.isPlaying == 1 else 0] == 2:
                        self.board[self.selected] = 0
                        self.moveNeeded = 1
                        self.checkerPositions[1 if self.isPlaying == 1 else 0].remove(self.selected)
                        if self.board in self.previusStates:
                            self.winner = 2
                            self.gamePhase = [3,3]
                    else:
                        self.inHand[1 if self.isPlaying == 1 else 0] -= 1
                        self.onBoard[1 if self.isPlaying == 1 else 0] += 1
                        if np.array(self.inHand).sum() == 0:
                            self.gamePhase = [1,1]
                            self.moveNeeded = 1
                    self.isPlaying = -self.isPlaying
            elif self.moveNeeded == 1: # choose checker to move
                if move in self.getValidMoves():
                    self.previusStates = np.append(self.previusStates, [copy.deepcopy(self.board)], axis=0)
                    valid = True
                    self.selected = move
                    if self.gamePhase[1 if self.isPlaying == 1 else 0] == 2:
                        self.moveNeeded = 0
                    else:
                        self.moveNeeded = 2
            elif self.moveNeeded == 2: # move checker up, down, left or right
                if self.getMoveFields(self.selected)[move] == 0:
                    idxToMoveAxis: np.ndarray = np.where(self.getInRows(self.selected) == self.selected)
                    idxToMove = list(zip(idxToMoveAxis[1], idxToMoveAxis[0]))
                    order = idxToMove[0][1]
                    valid = True
                    self.board[self.selected] = 0
                    last_state = self.getSummary(last_player)
                    self.checkerPositions[1 if self.isPlaying == 1 else 0].remove(self.selected)
                    if move == 0: # up
                        self.board[self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]-1]] = self.isPlaying
                        self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]-1])
                    if move == 1: # right
                        self.board[self.getInRows(self.selected)[0][idxToMove[order][0]+1]] = self.isPlaying
                        self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[0][idxToMove[order][0]+1])
                    if move == 2: # down
                        self.board[self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]+1]] = self.isPlaying
                        self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[1][idxToMove[abs(order-1)][0]+1])
                    if move == 3: # left
                        self.board[self.getInRows(self.selected)[0][idxToMove[order][0]-1]] = self.isPlaying
                        self.checkerPositions[1 if self.isPlaying == 1 else 0].append(self.getInRows(self.selected)[0][idxToMove[order][0]-1])
                    self.selected = -1
                    self.moveNeeded = 1
                    self.isPlaying = -self.isPlaying
                    if (self.board == self.previusStates).all(axis=1).any():
                        self.winner = 2
                        self.gamePhase = [3,3]
            elif self.moveNeeded == 3: # delete opponent checker
                reward = 1
                threeInChosenRow: np.ndarray = abs(self.board[self.getInRows(move)].sum(axis=1)) == 3
                if self.board[move] == -1 * self.isPlaying and ~threeInChosenRow.any():
                    valid = True
                    self.board[move] = 0
                    self.checkerPositions[1 if self.isPlaying == -1 else 0].remove(move)
                    self.onBoard[1 if self.isPlaying == -1 else 0] -= 1
                    self.previusStates = np.array([np.zeros(24)])
                    if self.gamePhase[1 if self.isPlaying == -1 else 0] == 0:
                        self.moveNeeded = 0
                    elif self.gamePhase[1 if self.isPlaying == -1 else 0] == 1:
                        if self.onBoard[1 if self.isPlaying == -1 else 0] == 3:
                            self.gamePhase[1 if self.isPlaying == -1 else 0] = 2
                        self.moveNeeded = 1
                    elif self.gamePhase[1 if self.isPlaying == -1 else 0] == 2:
                        self.gamePhase = [3,3]
                        self.winner = last_player
                    self.isPlaying = -self.isPlaying
            if last_state[0] < self.getSummary(last_player)[0]:
                self.isPlaying = last_player
                self.moveNeeded = 3
                canDelete: bool = False
                for pos in self.checkerPositions[1 if self.isPlaying == -1 else 0]:
                    if ~(abs(self.board[self.getInRows(pos)].sum(axis=1)) == 3).any():
                        canDelete = True
                        break
                if not canDelete:
                    valid = True
                    self.isPlaying = -self.isPlaying
                    if self.gamePhase[1 if self.isPlaying == -1 else 0] == 0:
                        self.moveNeeded = 0
                    elif self.gamePhase[1 if self.isPlaying == -1 else 0] >= 1:
                        self.moveNeeded = 1
            if self.gamePhase[1 if last_player == -1 else 0] == 1:
                finished = True
                for pos in self.checkerPositions[1 if last_player == -1 else 0]:
                    if ~(self.getMoveFields(pos).all()):
                        finished = False
                        break
                if finished:
                    self.winner = last_player
                    self.gamePhase = [3,3]
            return valid, reward
        return False, 0
    def isFinished(self):
        return self.winner
    def getBoard(self) -> np.ndarray:
        return self.board
    def getInRows(self, pos: int) -> np.ndarray:
        arrayPos: np.ndarray = self.columns == pos
        return self.columns[arrayPos.any(axis=1)]
    def reset(self):
        self.isPlaying: int = 1
        self.gamePhase: list = [0,0]
        self.moveNeeded: int = 0
        self.inHand: list = [9,9]
        self.onBoard: list = [0,0]
        self.checkerPositions: list = [[],[]]
        self.selected: int = -1
        self.board: np.ndarray = np.zeros(24)
        self.winner = 0
    def getSummary(self, player: int) -> (int, int):
        columnSums:np.ndarray = self.board[self.columns].sum(axis=1)
        numThreePlayerOpponent = np.count_nonzero(columnSums == -3 * player)
        numThreePlayerActual = np.count_nonzero(columnSums == 3 * player)
        return numThreePlayerActual, numThreePlayerOpponent
    def getMoveFields(self, pos: int) -> np.ndarray:
        moveFields: np.ndarray = np.zeros(4)
        chosenRows: np.ndarray = self.getInRows(pos)
        idxAxis: np.ndarray = np.where(chosenRows == pos)
        idx = list(zip(idxAxis[1], idxAxis[0]))
        order = idx[0][1]
        if idx[order][0] != 1:
            if idx[order][0] == 0:
                moveFields[3] = 2
                moveFields[1] = self.board[chosenRows[0][1]]
            elif idx[order][0] == 2:
                moveFields[1] = 2
                moveFields[3] = self.board[chosenRows[0][1]]
        else:
            moveFields[3] = self.board[chosenRows[0][0]]
            moveFields[1] = self.board[chosenRows[0][2]]
        if idx[abs(order-1)][0] != 1:
            if idx[abs(order-1)][0] == 0:
                moveFields[0] = 2
                moveFields[2] = self.board[chosenRows[1][1]]
            elif idx[abs(order-1)][0] == 2:
                moveFields[2] = 2
                moveFields[0] = self.board[chosenRows[1][1]]
        else:
            moveFields[0] = self.board[chosenRows[1][0]]
            moveFields[2] = self.board[chosenRows[1][2]]
        return moveFields
    def getValidMoves(self) -> List[int]:
        validList = []
        if self.moveNeeded == 0:
            validList = np.arange(24)
            validList = validList[self.board.flat == 0]
        elif self.moveNeeded == 1:
            validList = [pos for pos in self.checkerPositions[1 if self.isPlaying == 1 else 0] if ~self.getMoveFields(pos).all() or self.gamePhase[1 if self.isPlaying == 1 else 0] == 2]
        elif self.moveNeeded == 2:
            moveFields = self.getMoveFields(self.selected)
            validList = [move for move in np.arange(4) if moveFields[move] == 0]
        elif self.moveNeeded == 3:
            validList = [pos for pos in self.checkerPositions[1 if self.isPlaying == -1 else 0] if ~(abs(self.board[self.getInRows(pos)].sum(axis=1)) == 3).any()]
        return validList
    def getFullState(self):
        return self.getBoard(), self.isPlaying, self.gamePhase, self.moveNeeded,self.inHand, self.onBoard, self.checkerPositions, self.selected, self.winner, self.previusStates
    def setFullState(self, board, isPlaying, gamePhase, moveNeded, inHand, onboard, checkerPositions, selected, winner, previusStates):
        self.isPlaying: int = isPlaying
        self.gamePhase: list = copy.deepcopy(gamePhase)
        self.moveNeeded: int = moveNeded
        self.inHand: list = copy.deepcopy(inHand)
        self.onBoard: list = copy.deepcopy(onboard)
        self.checkerPositions: list = copy.deepcopy(checkerPositions)
        self.selected: int = selected
        self.board: np.ndarray = copy.deepcopy(board)
        self.winner: int = winner
        self.previusStates = copy.deepcopy(previusStates)

#### Agent

In [33]:
class Agent(object):
    def __init__(self):
        self.encoder = pre.OneHotEncoder(sparse=False).fit(np.array([1,0,-1]).reshape(-1,1))
    def processState(self, player, state, moveNeeded, selected= None):
        state = state * player
        state_OH = np.array(self.encoder.transform(state.reshape(-1,1)))
        if moveNeeded == 2 and selected is not None:
            state[selected] = 2
            selectedArray = np.zeros(24)
            selectedArray[selected] = 1
            # TODO append selectedArray to state_OH
            state_OH = np.append(state_OH, selectedArray.reshape(-1,1), axis=1)
        return state_OH.flatten()
        #return state
    def getPos(self, state: np.ndarray, temp: float, validMoves: List[int],network:keras.Model=None) -> np.ndarray:
        if network is None:
            return np.random.choice(validMoves)
        softmaxed_output = keras.backend.softmax(network(state.reshape(1,-1))/ temp)
        action_value = np.random.choice(np.array(softmaxed_output[0]), p= np.array(softmaxed_output[0]))
        pos: np.ndarray = np.argmax(softmaxed_output[0] == action_value)
        return pos


#### Graphics

In [34]:
class MillDisplayer(object):
    def __init__(self, MillEnvironment: MillEnv = None):
        psGui.theme("dark")
        self.millImage: str = "MühleBrett.png"
        self.blackCheckerImage: str  = "Schwarz.png"
        self.whiteCheckerImage: str = "Weiss.png"
        self.millEnv: MillEnv = MillEnv()
        if MillEnvironment is not None :
            self.millEnv = MillEnvironment
        self.ImageIDArray = np.array([])
        self.imageLocations = [(10,490), (225, 490), (440, 490),
                                (75,415), (225, 415), (375, 415),
                                (150,340), (225, 340), (310, 340),
                                (10,265), (75, 265), (150, 265),
                                (310,265), (375, 265), (440, 265),
                                (150,190), (225, 190), (310, 190),
                                (75,115), (225, 115), (375, 115),
                                (10,55), (225, 55), (440, 55)]
        self.graph = psGui.Graph(
                        canvas_size=(500, 500),
                        graph_bottom_left=(0, 0),
                        graph_top_right=(500, 500),
                        )
        self.statusTextBox = psGui.Text("Player "+self.getPlayerName(self.millEnv.isPlaying)+" is playing", size=(50, 1))
        self.layout_ = [[psGui.Button("Player vs. Player"),psGui.Button("Player vs. Agent"),psGui.Button("Agent vs. Agent")],
                        [self.statusTextBox],
                       [self.graph],
                       [psGui.Button("Close")]]
        self.window  = psGui.Window("Mill AI", layout=self.layout_, finalize=True)
        self.window.finalize()
        self.graph.DrawImage(filename=self.millImage, location=(0,500))
        self.activateClick()
        self.reloadEnv()
    def windowsLoop(self):
        while True:
            event, values = self.window.read()
            if event == psGui.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
                break
            elif not event == "":
                self.reset()
        self.window.close()
    def makeMove(self, pos: int) -> bool:
        valid, reward = self.millEnv.makeMove(pos)
        if valid:
            self.reloadEnv()
        return valid
    def reloadEnv(self):
        self.setStatus("Player " + self.getPlayerName(self.millEnv.isPlaying) + " is playing - move needed: " + str(self.millEnv.moveNeeded))
        for imageID in self.ImageIDArray:
            self.graph.DeleteFigure(imageID)
        np.delete(self.ImageIDArray, np.s_[:])
        for case, location in zip(self.millEnv.getBoard(), self.imageLocations):
            if case == 1:
                self.ImageIDArray = np.append(self.ImageIDArray,self.graph.DrawImage(filename=self.blackCheckerImage, location=location))
            elif case == -1:
                self.ImageIDArray = np.append(self.ImageIDArray,self.graph.DrawImage(filename=self.whiteCheckerImage, location=location))
        self.window.refresh()
    def getClicked(self, event) -> int:
        for index, location in enumerate(self.imageLocations):
            x2, y2 = location
            if self.isInArea(event.x, -event.y + 500, x2, y2, 50, 50):
                return index
        return -1
    def setAfterClicked(self, event):
        pos = self.getClicked(event)
        if pos == -1:
            return False
        if self.millEnv.moveNeeded == 2:
            dif = self.millEnv.selected - pos
            if dif == 0:
                return False
            if dif == -1:
                pos = 1
            elif dif == 1:
                pos = 3
            elif dif < 0:
                pos = 2
            elif dif > 0:
                pos = 0
        return self.makeMove(pos)
    def isInArea(self, posX1: int, posY1: int, posX2: int, posY2: int, width: int, height: int) -> bool:
        if posX2 <= posX1 <= posX2 + width:
            if posY2 >= posY1 >= posY2 - height:
                return True
        return False
    def setStatus(self, status: str):
        self.statusTextBox.Update(status)
    def close(self):
        self.window.close()
    def activateClick(self):
        self.graph.TKCanvas.bind("<Button-1>",self.setAfterClicked)
    def deactivateClick(self):
        self.graph.TKCanvas.unbind("<Button-1>")
    def read(self, timout: bool=False):
        return self.window.read(0 if timout else None)
    def reset(self):
        self.millEnv.reset()
        self.reloadEnv()
    def getPlayerName(self, player: int) -> str:
        if player == 1:
            return "black"
        elif player == -1:
            return "white"
        else:
            return "not a player"


#### Moderated Graphics

In [35]:
class ModeratedGraphics(object):
    def __init__(self, gamma=0.9, num_sims=250, max_depth=15):
        self.max_depth = max_depth
        self.num_sims = num_sims
        self.gamma = gamma
        self.env = MillEnv()
        self.graphics = MillDisplayer(self.env)
        self.graphics.reloadEnv()
        self.root: State = State(self.env)
        self.mcts = MonteCarloTreeSearch(self.root)
    def agentPlay(self):
        self.resetMonteCarlo()
        self.graphics.deactivateClick()
        finished = 0
        while finished == 0:
            self.graphics.reloadEnv()
            pos = self.mcts.best_action(self.gamma,multiplikator=self.num_sims, max_depth=self.max_depth)
            self.mcts.setNewRoot(State(self.env))
            self.graphics.makeMove(pos)
            event, values = self.graphics.read(True)
            if self.eventHandler(event):
                return
            finished = self.env.isFinished()
        self.graphics.reloadEnv()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
    def playersVSPlayer(self):
        self.graphics.activateClick()
        self.graphics.reset()
        finished = 0
        while finished == 0:
            event, values = self.graphics.read()
            if self.eventHandler(event):
                return
            self.graphics.reloadEnv()
            finished = self.env.isFinished()
        self.graphics.reloadEnv()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
        self.graphics.deactivateClick()
    def playerVSAgent(self):
        self.graphics.activateClick()
        self.resetMonteCarlo()
        finished = 0
        while finished == 0:
            event, values = self.graphics.read(True)
            if self.eventHandler(event):
                return
            if self.env.isPlaying == 1:
                self.graphics.activateClick()
            else:
                self.graphics.deactivateClick()
                self.root = State(self.env)
                self.mcts.setNewRoot(self.root)
                pos = self.mcts.best_action(self.gamma,multiplikator=self.num_sims, max_depth=self.max_depth)
                self.mcts.setNewRoot(State(self.env))
                self.graphics.makeMove(pos)
            self.graphics.reloadEnv()
            finished = self.env.isFinished()
        self.graphics.reloadEnv()
        if not finished == 2:
            self.graphics.setStatus("player " + self.graphics.getPlayerName(finished) +" won")
        else:
            self.graphics.setStatus("The game ended in a draw")
        self.graphics.deactivateClick()
    def playLoop(self):
        self.graphics.deactivateClick()
        self.playerVSAgent()
        finished = False
        while not finished:
            event, values = self.graphics.read()
            finished = self.eventHandler(event)
    def eventHandler(self, event) -> bool:
        if event == psGui.WIN_CLOSED or event == 'Close': # if user closes window or clicks cancel
            self.graphics.close()
            return True
        elif event == "Agent vs. Agent":
            self.agentPlay()
        elif event == "Player vs. Player":
            self.playersVSPlayer()
        elif event == "Player vs. Agent":
            self.playerVSAgent()
        return False
    def resetMonteCarlo(self):
        self.env.reset()
        self.root = State(self.env)
        self.mcts.setNewRoot(self.root)

#### Memory

In [36]:
class Memory(object):
    def __init__(self, size: int):
        self.size = size
        self.curr_write_idx = 0
        self.available_samples = 0
        self.buffer = np.array([(np.zeros(24,dtype=np.float32), 0.0, 0.0, np.zeros(24,
                                dtype=np.float32), False) for index in range(self.size)], dtype=object)
        self.base_node, self.leaf_nodes = create_tree([0 for index in range(self.size)])
        self.frame_idx = 0
        self.action_idx = 1
        self.reward_idx = 2
        self.terminal_idx = 3
        self.beta = 0.4
        self.alpha = 0.6
        self.min_priority = 0.01

    def append(self, experience: tuple, priority: float):
        self.buffer[self.curr_write_idx] = experience
        self.update(self.curr_write_idx, priority)
        self.curr_write_idx += 1
        # reset the current writer position index if creater than the allowed size
        if self.curr_write_idx >= self.size:
            self.curr_write_idx = 0
        # max out available samples at the memory buffer size
        if self.available_samples + 1 < self.size:
            self.available_samples += 1
        else:
            self.available_samples = self.size - 1

    def update(self, idx: int, priority: float):
        update(self.leaf_nodes[idx], self.adjust_priority(priority))

    def adjust_priority(self, priority: float):
        return np.power(priority + self.min_priority, self.alpha)

    def sample(self, num_samples: int):
        sampled_idxs = []
        is_weights = []
        sample_no = 0
        while sample_no < num_samples:
            sample_val = np.random.uniform(0, self.base_node.value)
            samp_node = retrieve(sample_val, self.base_node)
            if samp_node.idx < self.available_samples - 1:
                sampled_idxs.append(samp_node.idx)
                p = samp_node.value / self.base_node.value
                is_weights.append((self.available_samples + 1) * p)
                sample_no += 1
        # apply the beta factor and normalise so that the maximum is_weight < 1
        is_weights = np.array(is_weights)
        is_weights = np.power(is_weights, -self.beta)
        is_weights = is_weights / np.max(is_weights)
        # now load up the state and next state variables according to sampled idxs
        return self.buffer[sampled_idxs], sampled_idxs, is_weights


#### Model

In [37]:
class Residual_Model(keras.Model):
    def __init__(self, hidden_size: int, num_actions: int, input_shape: (int, int, int) = None, input_tensor: tf.Tensor = None):
        super(Residual_Model, self).__init__()
        self.residual = keras.applications.resnet_v2.ResNet50V2(include_top=False, weights=None, input_shape=input_shape, input_tensor=input_tensor, pooling='max')
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(hidden_size *3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense3 = keras.layers.Dense(hidden_size * 5, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.dense4 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense1 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_dense3 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.adv_out = keras.layers.Dense(num_actions, activation='softmax',
                                          kernel_initializer=keras.initializers.he_normal())
        self.v_dense1 = keras.layers.Dense(hidden_size * 3, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.v_dense2 = keras.layers.Dense(hidden_size * 4, activation='relu',
                                         kernel_initializer=keras.initializers.he_normal())
        self.v_out = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.he_normal())

    def call(self, input, **kwargs):
        x = self.residual(input)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        adv = self.adv_dense1(x)
        adv = self.adv_dense2(adv)
        adv = self.adv_dense3(adv)
        adv = self.adv_out(adv)
        v = self.v_dense1(x)
        v = self.v_dense2(v)
        v = self.v_out(v)
        return adv, v

    @tf.function
    def traceable(self, input, **kwargs):
        return self(input, **kwargs)

#### TreeNode

In [38]:
class Node:
    def __init__(self, left, right, is_leaf: bool = False, idx = None):
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.value = sum(n.value for n in (left, right) if n is not None)
        self.parent = None
        self.idx = idx  # this value is only set for leaf nodes
        if left is not None:
            left.parent = self
        if right is not None:
            right.parent = self

    @classmethod
    def create_leaf(cls, value, idx):
        leaf = cls(None, None, is_leaf=True, idx=idx)
        leaf.value = value
        return leaf


def create_tree(input: list):
    nodes = [Node.create_leaf(v, i) for i, v in enumerate(input)]
    leaf_nodes = nodes
    while len(nodes) > 1:
        inodes = iter(nodes)
        nodes = [Node(*pair) for pair in zip(inodes, inodes)]

    return nodes[0], leaf_nodes

def retrieve(value: float, node: Node):
    if node.is_leaf:
        return node

    if node.left.value >= value:
        return retrieve(value, node.left)
    else:
        return retrieve(value - node.left.value, node.right)

def update(node: Node, new_value: float):
    change = new_value - node.value

    node.value = new_value
    propagate_changes(change, node.parent)


def propagate_changes(change: float, node: Node):
    node.value += change

    if node.parent is not None:
        propagate_changes(change, node.parent)

#### State

In [46]:
class State(object):
    def __init__(self, env: MillEnv, last_move = None, parent = None, seed = None):
        self.last_move = last_move
        self.n = 0
        self.q = 0
        self.terminal = env.isFinished()
        self.valid_moves = env.getValidMoves()
        self.untried_actions: list = list(self.valid_moves)
        self.state = env.getFullState()
        self.parent = parent
        self.children = np.array([])
        self.results = defaultdict(float)
        self.env: MillEnv = env
        self.move_value = 0
        self.random_gen = np.random.Generator(np.random.PCG64(seed))
    def is_fully_expanded(self):
        return len(self.untried_actions) == 0
    def best_child(self, c_param=1.):
        choices_weights = [
            (c.q / c.n) + c_param * np.sqrt((2 * np.log(self.n) / c.n))
            for c in self.children
        ]
        return self.children[np.argmax(choices_weights)]
    def rollout_policy(self):
        return self.valid_moves[np.random.randint(len(self.valid_moves))]
    def is_terminal_node(self):
        return self.terminal
    def expand(self):
        action = self.untried_actions.pop()
        self.env.setFullState(self.state[0], self.state[1],self.state[2], self.state[3],self.state[4], self.state[5],self.state[6], self.state[7], self.state[8], self.state[9])
        self.env.makeMove(action)
        child_node = State(self.env,action, parent=self)
        self.children = np.append(self.children, child_node)
        return child_node
    def rollout(self, discount, max_depth = 20, root = None):
        if self.parent is not None:
            self.env.setFullState(self.parent.state[0], self.parent.state[1],self.parent.state[2], self.parent.state[3],self.parent.state[4], self.parent.state[5],self.parent.state[6], self.parent.state[7], self.parent.state[8], self.parent.state[9])
        else: 
            self.env.setFullState(self.state[0], self.state[1],self.state[2], self.state[3],self.state[4], self.state[5],self.state[6], self.state[7], self.state[8], self.state[9])
        reward = defaultdict(float)
        depth = max_depth - self.depth(root)
        for iteration in range(depth):
            last_player = self.env.isPlaying
            if iteration == 0 and self.last_move:
                action = self.last_move
            else:
                action = self.random_gen.choice(self.env.getValidMoves())
            if iteration == 0:
                self.move_value = self.env.makeMove(action)[1]
            else:
                reward[last_player] += self.env.makeMove(action)[1]
            if self.env.isFinished() != 0:
                depth = iteration
                break
        if self.parent:
            if self.env.isFinished() == self.parent.state[1]:
                reward[self.parent.state[1]] += discount**depth *5
            elif self.env.isFinished() == -self.parent.state[1]:
                reward[-self.parent.state[1]] += discount**depth *5
            elif self.env.isFinished() == 2:
                reward[self.parent.state[1]] -= 0.5
        return reward
    def depth(self, root):
        depth = 0
        current_parent = self
        while current_parent.parent != root and current_parent.parent:
            current_parent = current_parent.parent
            depth += 1
        return depth
    def backpropagate(self, result):
        current_node = self
        iters = 1
        while current_node.parent:
            current_node.n += 1
            result[current_node.parent.state[1]] += current_node.move_value/iters
            current_node.q += result[current_node.parent.state[1]]
            current_node.q -= result[-current_node.parent.state[1]]
            current_node = current_node.parent
            iters += 1
        current_node.n +=1
    def mergeChildren(self, states: np.ndarray):
        self.q = np.sum([state.q for state in states])
        self.n = np.sum([state.n for state in states])
        childrenAll: np.ndarray = np.array([state.children for state in states], dtype= object).transpose()
        for children in childrenAll:
            valididx = ~(children == None)
            if len(valididx) >= 1:
                self.children = np.append(self.children, children[valididx].reshape(-1)[0].merge(children[valididx]))
        return self
    def merge(self, children):
        self.q = np.sum([state.q for state in children])
        self.n = np.sum([state.n for state in children])
        return self


#### Monte Carlo Tree Search

In [40]:
class MonteCarloTreeSearch(object):
    def __init__(self, node):
        self.root: State = node
    def best_action(self, gamma, parallel = mp.cpu_count()//2, multiplikator = 250, max_depth = 20) -> int:
        """
        get best action of state
        Parameters
        -------
        :parameter gamma: the discount factor
        :parameter max_depth: depth to search actions
        :parameter multiplikator : number of simulations performed to get the best action
        Returns
        -------
        :returns best child state
        """
        if __name__ == '__main__':
            with mp.Pool(parallel) as pool:
                result = pool.starmap(self.train, [(gamma, multiplikator,max_depth) for _ in range(parallel)])
        self.root.mergeChildren(np.array(result))
        # to select best child go for exploitation only
        self.root.env.setFullState(self.root.state[0],self.root.state[1],self.root.state[2],self.root.state[3],self.root.state[4],self.root.state[5],self.root.state[6],self.root.state[7],self.root.state[8], self.root.state[9])
        return self.root.best_child(c_param=0.).last_move
    def train(self, gamma, multiplikator = 500, max_depth = 20):
        """
        get best action of state
        Parameters
        -------
        :parameter gamma: the discount factor
        :parameter max_depth: depth to search actions
        :parameter multiplikator : number of simulations performed to get the best action
        Returns
        -------
        :returns best child state
        """
        self.root.random_gen = np.random.Generator(np.random.PCG64())
        simulations_number = int(len(self.root.valid_moves)**1.2 * multiplikator)
        for i in range(simulations_number):
            v = self._tree_policy(max_depth)
            reward = v.rollout(gamma, max_depth, self.root)
            v.backpropagate(reward)
        for child in self.root.children:
            child.children = np.array([])
        return self.root


    def setNewRoot(self, node):
        self.root = node
    def _tree_policy(self, max_depth = 200, expl = 1.4):
        """
        selects node to run rollout/playout for
        Returns
        -------
        """

        current_node = self.root
        itern  = 0
        while not current_node.is_terminal_node() and itern < max_depth:
            if not current_node.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child(expl)
            itern += 1
        return current_node


### Tests

#### Environment

In [41]:
envis = MillEnv()
print(envis.getInRows(14))

envis.makeMove(0)
envis.makeMove(3)
envis.makeMove(2)
envis.makeMove(4)
envis.makeMove(1)
envis.makeMove(3)
envis.makeMove(3)
print(envis.getValidMoves())
print(envis.getBoard())
print(envis.moveNeeded)
ThreeInChosenRow = abs(envis.board[envis.getInRows(0)].sum(axis=1)) == 3
print(~ThreeInChosenRow.any())
print(envis.getMoveFields(4))
print(envis.getSummary(1))

[[12 13 14]
 [ 2 14 23]]
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 1.  1.  1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]
0
False
[ 1.  0.  0. -1.]
(1, 0)


#### Agent

In [109]:
envi = MillEnv()
ag = Agent()
print(ag.processState(1,envi.getBoard(), 2,3).shape)
firstTime = time.time()
iters = 10
for i in range(iters):
    envi.reset()
    while envi.isFinished() == 0:
        validNow = envi.makeMove(ag.getPos(envi.getBoard(), 0, envi.getValidMoves()))
        if not validNow:
            print("invalid")
    print(f"Gewonnen hat {envi.isFinished()}")
print(time.time()- firstTime)
print((time.time()-firstTime)/iters)

(96,)
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
Gewonnen hat 2
0.6103560924530029
0.06104590892791748


#### merge

In [ ]:
mcts = MonteCarloTreeSearch(State(MillEnv()))
%time print(mcts.best_action(0.99, multiplikator=750, max_depth=12))

#### Displayer

In [44]:
displayer = MillDisplayer()
displayer.windowsLoop()

#### Monte Carlo

In [ ]:
MCGraphics = ModeratedGraphics(gamma=0.9, max_depth=12, num_sims=750)
MCGraphics.playLoop()